# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [69]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import pprint
# Import API key
from api_keys import geoapify_key
geoapify_key

'6fdc166bf4eb4bf8874e18692e9b3b6d'

In [70]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,rio grande,-32.0350,-52.0986,298.12,89,68,1.90,BR,1708679562
1,1,bredasdorp,-34.5322,20.0403,289.77,82,10,2.82,ZA,1708748534
2,2,jalalaqsi,3.3766,45.5996,297.63,66,26,7.50,SO,1708744203
3,3,la passe,45.5549,-0.8967,278.28,77,71,5.44,FR,1708757532
4,4,alice springs,-23.7000,133.8833,307.71,22,0,5.14,AU,1708721815


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [71]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

city_map=city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo= True,
    color='City',
    tiles='CartoDark',
    size='Humidity',
    scale=0.5)

# Display the map
city_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [64]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

ideal_city_df = city_data_df.loc[(city_data_df['Max Temp'] > 60) & (city_data_df['Max Temp'] < 250)]


# Drop any rows with null values
# YOUR CODE HERE

ideal_city_df.dropna()

# Display sample data
# YOUR CODE HERE
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,susuman,62.7833,148.1667,247.75,92,98,1.24,RU,1708723906
28,28,iqaluit,63.7506,-68.5145,242.00,69,40,3.09,CA,1708689782
49,49,askiz,53.1319,90.5264,247.97,97,100,0.65,RU,1708736322
89,89,udachny,66.4167,112.4000,249.09,73,100,1.01,RU,1708733274
96,96,ust-nera,64.5667,143.2000,245.61,97,95,1.43,RU,1708725462
108,108,bilibino,68.0546,166.4372,245.89,94,100,1.29,RU,1708720793
130,130,zheleznogorsk-ilimskiy,56.5768,104.1217,249.70,96,100,2.53,RU,1708733475
204,204,tazovsky,67.4667,78.7000,249.30,98,70,4.90,RU,1708741620
239,239,kokshetau,53.2775,69.3883,243.90,95,29,2.30,KZ,1708741404
288,288,suntar,62.1444,117.6319,249.18,100,100,1.23,RU,1708731098


### Step 3: Create a new DataFrame called `hotel_df`.

In [65]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_city_df[['City','Country','Lat','Lng','Humidity']].copy()

hotel_df["Hotel Name"] = " "


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = " "


# Display sample data
# YOUR CODE HERE

hotel_df.head()





,City,Country,Lat,Lng,Humidity,Hotel Name
14,susuman,RU,62.7833,148.1667,92,
28,iqaluit,CA,63.7506,-68.5145,69,
49,askiz,RU,53.1319,90.5264,97,
89,udachny,RU,66.4167,112.4000,73,
96,ust-nera,RU,64.5667,143.2000,97,


In [66]:
# Set parameters to search for a hotel
radius = 10000
limit  = 1

Lng = 61.20
Lat = 52.19
categories = "accommodation.hotel"
filters = f"circle:{Lng},{Lat},{radius}"
bias=f"proximity:{Lng},{Lat}"

print(filters)
print(bias)
PARAMS = {
    "categories":categories,
    "bias":bias,
    "filter":filters,
    "apiKey":geoapify_key
}

base_url = f"https://api.geoapify.com/v2/places?"

name_address = requests.get(base_url, params=PARAMS).json()
#pprint.pprint(name_address)

json_formatted_name_address = json.dumps(name_address, indent=2)
print(json_formatted_name_address)
hotel_name = name_address['features'][0]['properties']['name']
hotel_name

circle:61.2,52.19,10000
proximity:61.2,52.19
{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {
        "name": "\u0433\u043e\u0441\u0442\u0438\u043d\u0438\u0446\u0430 \"\u041e\u043a\u0442\u044f\u0431\u0440\u044c\"",
        "country": "Kazakhstan",
        "country_code": "kz",
        "state": "Kostanay Region",
        "county": "Jitiqara district",
        "city": "\u0416\u0456\u0442\u0456\u049b\u0430\u0440\u0430",
        "postcode": "110700",
        "street": "\u0443\u043b\u0438\u0446\u0430 \u0428\u0430\u0445\u0442\u0435\u0440\u043e\u0432",
        "housenumber": "4",
        "lon": 61.2175214,
        "lat": 52.1856485,
        "formatted": "\u0433\u043e\u0441\u0442\u0438\u043d\u0438\u0446\u0430 \"\u041e\u043a\u0442\u044f\u0431\u0440\u044c\", Kazakhstan, Kostanay Region, 110700 \u0416\u0456\u0442\u0456\u049b\u0430\u0440\u0430, \u0443\u043b\u0438\u0446\u0430 \u0428\u0430\u0445\u0442\u0435\u0440\u043e\u0432, 4",
        "address_

'гостиница "Октябрь"'

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [67]:
# Set parameters to search for a hotel
radius = 10000
limit  = 1

radius = 10000
limit  = 1

categories = "accommodation.hotel"
filters = ""
bias=""

print(filters)
print(bias)
PARAMS = {
    "categories":categories,
    "bias":bias,
    "filter":filters,
    "lang":"en",
    "apiKey":geoapify_key
}

base_url = f"https://api.geoapify.com/v2/places?"

name_address = requests.get(base_url, params=PARAMS).json()
#pprint.pprint(name_address)

#json_formatted_name_address = json.dumps(name_address, indent=2)
#print(json_formatted_name_address)
#hotel_name = name_address['features'][0]['properties']['name']
#hotel_name


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    Lat = row['Lat']
    Lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    PARAMS["filter"] = f"circle:{Lng},{Lat},{radius}"
    PARAMS["bias"] = f"proximity:{Lng},{Lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=PARAMS).json()
    
    # Convert the API response to JSON format
   # name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df



Starting hotel search
susuman - nearest hotel: No hotel found
iqaluit - nearest hotel: Frobisher Inn
askiz - nearest hotel: No hotel found
udachny - nearest hotel: Вилюй
ust-nera - nearest hotel: гостиница "Солнечная"
bilibino - nearest hotel: No hotel found
zheleznogorsk-ilimskiy - nearest hotel: Магнетит
tazovsky - nearest hotel: Тазовчанка
kokshetau - nearest hotel: Отель Green Which
suntar - nearest hotel: No hotel found
ust'-ordynskiy - nearest hotel: Байкал
severo-yeniseyskiy - nearest hotel: Актолик
chernyshevskiy - nearest hotel: No hotel found
amga - nearest hotel: "АммА"
la sarre - nearest hotel: Motel le Bivouac
arkalyk - nearest hotel: Гостиница "Арсенал"
vilyuysk - nearest hotel: Мария
tulun - nearest hotel: Центральная
khandyga - nearest hotel: Апельсин
chunskiy - nearest hotel: Теремок
namtsy - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
14,susuman,RU,62.7833,148.1667,92,No hotel found
28,iqaluit,CA,63.7506,-68.5145,69,Frobisher Inn
49,askiz,RU,53.1319,90.5264,97,No hotel found
89,udachny,RU,66.4167,112.4000,73,Вилюй
96,ust-nera,RU,64.5667,143.2000,97,"гостиница ""Солнечная"""
108,bilibino,RU,68.0546,166.4372,94,No hotel found
130,zheleznogorsk-ilimskiy,RU,56.5768,104.1217,96,Магнетит
204,tazovsky,RU,67.4667,78.7000,98,Тазовчанка
239,kokshetau,KZ,53.2775,69.3883,95,Отель Green Which
288,suntar,RU,62.1444,117.6319,100,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [68]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
city_plot = hotel_df.hvplot.scatter(x="Lng", y="Lat", size="Humidity", color="Blue",
                                   hover_cols=["City", "Country", "Hotel Name"],
                                   alpha=0.5, title="City Weather Data with Hotels")

# Display the map
city_plot
# Display the map
# YOUR CODE HERE

:Scatter   [Lng]   (Lat,Humidity,City,Country,Hotel Name)